In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\\NonSystem\\Work\\mail\\'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

C:\\NonSystem\\Work\\mail\\answers.jsonl
C:\\NonSystem\\Work\\mail\\main_category_mapper.json
C:\\NonSystem\\Work\\mail\\russian-dictionary-master.zip
C:\\NonSystem\\Work\\mail\\sample_submission.csv
C:\\NonSystem\\Work\\mail\\sub_category_mapper.json
C:\\NonSystem\\Work\\mail\\test.csv
C:\\NonSystem\\Work\\mail\\train.csv
C:\\NonSystem\\Work\\mail\\unsupervised.csv
C:\\NonSystem\\Work\\mail\\zaliznjak_forms.txt
C:\\NonSystem\\Work\\mail\\russian-dictionary-master\adjectives.csv
C:\\NonSystem\\Work\\mail\\russian-dictionary-master\LICENSE
C:\\NonSystem\\Work\\mail\\russian-dictionary-master\nouns.csv
C:\\NonSystem\\Work\\mail\\russian-dictionary-master\others.csv
C:\\NonSystem\\Work\\mail\\russian-dictionary-master\README.md
C:\\NonSystem\\Work\\mail\\russian-dictionary-master\verbs.csv


In [26]:
train = pd.read_csv(r'C:\\NonSystem\\Work\\mail\\train.csv')
test = pd.read_csv(r'C:\\NonSystem\\Work\\mail\\test.csv')
unsupervised = pd.read_csv(r'C:\\NonSystem\\Work\\mail\\unsupervised.csv')

In [27]:
unsupervised.insert(0,'main_category',['' for i in range(len(unsupervised))])

In [28]:
unsupervised.insert(1,'sub_category',['' for i in range(len(unsupervised))])

In [29]:
# A dictionary to map a Russian word with its lemma.
lem_dict = {}
with open(r'C:\\NonSystem\\Work\\mail\\zaliznjak_forms.txt',encoding='utf-8-sig') as dictionary:
    dictionary = dictionary.read()
    dictionary = dictionary.split('\n')
    for i in dictionary:
        i = i.split(',')
        try:
            word = i[0].replace("'", "")
            lemma = i[1]
            lem_dict[word] = lemma
        except:
            pass

In [30]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [31]:
# Data augmentation. Reverse the sentences.
def data_aug(dataframe):
    
    aug_train = {'main_category':[],'sub_category':[],'question':[]}
    for index, content in dataframe.iterrows():
        main = content['main_category']
        sub = content['sub_category']
        ques = content['question']
        ques = ques.split(' ')
        ques = ques[::-1]
        ques = ' '.join(ques)
        aug_train['main_category'].append(main)
        aug_train['sub_category'].append(sub)
        aug_train['question'].append(ques)
    aug_train_pd = pd.DataFrame(aug_train)
    new_dataframe = dataframe.append(aug_train_pd, ignore_index=True)
    
    return new_dataframe

In [8]:
train = data_aug(train)

In [9]:
unsupervised = data_aug(unsupervised)

In [10]:
def process_text(text):
    
    words = wordpunct_tokenize(text.lower())
    
    return words

In [11]:
word2freq = {}

for question in tqdm(train.question):
    
    words = process_text(question)
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

100%|████████████████████████████████████████████████████████████████████| 1333332/1333332 [00:13<00:00, 102518.67it/s]


In [12]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open(r'C:\Users\gonge\cc.ru.300.vec',encoding='utf-8')
    
n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)



while True:

    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()

    current_word = ' '.join(current_parts[:-embedding_dim])

    if current_word in word2freq:

        word2index[current_word] = len(word2index)

        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)

        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

word2vec_file.close()

vectors = np.concatenate(vectors)

Read word2vec: 100%|██████████████████████████████████████████████████████| 2000000/2000000 [01:12<00:00, 27544.24it/s]


In [13]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

Мы не знаем 3.02 % слов в датасете
Количество неизвестных слов 107889 из 328896, то есть 32.80 % уникальных слов в словаре
В среднем каждое встречается 4.17 раз

Топ 5 невошедших слов:
??? с количеством вхождениий - 17208
!!! с количеством вхождениий - 13952
?) с количеством вхождениий - 13226
?? с количеством вхождениий - 12654
"? с количеством вхождениий - 9162


In [13]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=32, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
    
        words = wordpunct_tokenize(text.lower())
        words = [word for word in words if word.isalpha()] # We don't want non-word tokens.
        #words = [lem_dict[word] for word if word in lem_dict] #For lemmatization, which worsens the performance. So we don't use it here.
        
        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            words = self.process_text(text)
            indexed_words = self.indexing(words)
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):

        # здесь мы не используем токен UNK, потому что мы мы его специально не учили
        # становится непонятно какой же эмбеддинг присвоить неизвестному слову,
        # поэтому просто выбрасываем наши неизветсные слова
        
        ### CODE ###

        return [self.word2index[token] for token in tokenized_text if token in self.word2index ]
    
    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        
        ### CODE ###

        if len(sequence) > self.sequence_length:
            sequence = sequence[:self.sequence_length]
        elif len(sequence) < self.sequence_length:
            sequence = sequence + [self.pad_index] * (self.sequence_length - len(sequence))

        return sequence
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y

In [14]:
main_x_train, main_x_validation, main_y_train, main_y_validation = train_test_split(train.question, train.main_category, test_size=0.1)
sub_x_train, sub_x_validation, sub_y_train, sub_y_validation = train_test_split(train.question, train.sub_category, test_size=0.1)

main_train_dataset = WordData(list(main_x_train), list(main_y_train), word2index)
main_train_loader = DataLoader(main_train_dataset, batch_size=64)

sub_train_dataset = WordData(list(sub_x_train), list(sub_y_train), word2index)
sub_train_loader = DataLoader(sub_train_dataset, batch_size=64)

main_validation_dataset = WordData(list(main_x_validation), list(main_y_validation), word2index)
main_validation_loader = DataLoader(main_validation_dataset, batch_size=64)

sub_validation_dataset = WordData(list(sub_x_validation), list(sub_y_validation), word2index)
sub_validation_loader = DataLoader(sub_validation_dataset, batch_size=64)

main_test_dataset = WordData(list(test.question), np.zeros((test.shape[0])), word2index)
main_test_loader = DataLoader(main_test_dataset, batch_size=64)

sub_test_dataset = WordData(list(test.question), np.zeros((test.shape[0])), word2index)
sub_test_loader = DataLoader(sub_test_dataset, batch_size=64)

Loading data: 100%|█████████████████████████████████████████████████████████| 200000/200000 [00:02<00:00, 93787.47it/s]


In [15]:
main_x_unsu, main_x_unsu_vali, main_y_unsu, main_y_unsu_vali = train_test_split(unsupervised.question, unsupervised.main_category, test_size=0.00001)
sub_x_unsu, sub_x_unsu_vali, sub_y_unsu, sub_y_unsu_vali = train_test_split(unsupervised.question, unsupervised.sub_category, test_size=0.00001)
main_unsu_dataset = WordData(list(main_x_unsu), list(main_y_unsu), word2index)
main_unsu_loader = DataLoader(main_unsu_dataset, batch_size=64)
sub_unsu_dataset = WordData(list(sub_x_unsu), list(sub_y_unsu), word2index)
sub_unsu_loader = DataLoader(sub_unsu_dataset, batch_size=64)

Loading data: 100%|███████████████████████████████████████████████████████| 5070835/5070835 [00:58<00:00, 86983.32it/s]


In [16]:
class DeepAverageNetwork(torch.nn.Module):
    
    def __init__(self, embedding_matrix, n_classes):
        
        super().__init__()
        
        self.embedding_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(embedding_matrix))
        
        self.layers = torch.nn.Sequential(torch.nn.Linear(300, 256),
                                          torch.nn.ReLU(), 
                                          torch.nn.Linear(256, 128),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(128, n_classes))
    def forward(self, x):
        
        
        
        sequence_lengths = (x > 0).sum(dim=1)
        sequence_lengths[sequence_lengths == 0.] = 1
        
        x = self.embedding_layer(x)
        
        x = x.mean(dim=-2)
        
        lengths_scaling = sequence_lengths.float() / x.size(1)
        lengths_scaling = lengths_scaling.unsqueeze(1).repeat((1, x.size(-1)))
        x /= lengths_scaling.to(x.device)
        
        x = self.layers(x)
        
        return x

In [17]:
import torch.nn.functional as F
import torch.optim as optim

batch_size = 64
main_num_classes = 28
sub_num_classes = 209
n_hidden = 128
vocab_size = len(word2index)
embedding_dim  = 512

def attention(encode):

        attention_scores = torch.bmm(encode, encode.transpose(1, 2))
        attention_distribution = torch.softmax(attention_scores, 2)
        attention_vectors = torch.bmm(attention_distribution, encode)

        decoder_with_attention = torch.cat([encode, attention_vectors],dim=-1)
        
        return decoder_with_attention

class Model(nn.Module):
    def __init__(self,embedding_dim, n_hidden):
        super(Model, self).__init__()
        
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, n_hidden, bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        self.cnn = torch.nn.Conv1d(in_channels=300, out_channels=256, kernel_size=2)
        self.linear = torch.nn.Linear(in_features=256, out_features=128)
        self.pool = nn.AdaptiveAvgPool1d(1)
    
    def forward(self, X):
        X = self.embedding(X)
        X = self.dropout(X)
        X = X.transpose(0,1)
        X, states = self.lstm(X)
        X = X.transpose(0,1)
        X = self.dropout(X)
        X = attention(X)
        X = X.transpose(1,2)
        X = self.cnn(X)
        X = torch.relu(X)
        X = self.pool(X)
        X = X.squeeze()
        X = self.linear(X)
        
        
        
        return X



In [18]:
import torch.optim as optim

def sigmoid(d):
    np.sum(np.exp(d)/np.sum(np.exp(d)) *np.array([0,1,2,3,4]))


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')    

main_num_classes = 28
sub_num_classes = 209
main_mode = DeepAverageNetwork(embedding_matrix=vectors, n_classes=main_num_classes)
#main_mode = Model().to(device)  # complicated model

criterion = nn.CrossEntropyLoss()

sub_mode = DeepAverageNetwork(embedding_matrix=vectors, n_classes=sub_num_classes)
sub_mode = sub_mode.to(device)
#sub_mode = Model().to(device)
optimizer2 = optim.Adam(sub_mode.parameters(), lr=0.01)



main_mode = main_mode.to(device)

optimizer1 = optim.Adam(main_mode.parameters(), lr=0.01)

criterion = criterion.to(device)
# Training

epochs = 10
main_losses = []
sub_losses = []
best_test_loss = 10.

main_test_f1, sub_test_f1 = [], []

for n_epoch in tqdm(range(epochs)):
    
    main_train_losses = []
    main_test_losses = []
    main_test_targets = []
    main_test_pred_class = []
    sub_train_losses = []
    sub_test_losses = []
    sub_test_targets = []
    sub_test_pred_class = []
    
    sub_unsu_targets = []
    sub_unsu_pred_class = []
    main_unsu_targets = []
    main_unsu_pred_class = []
    main_chosen = []
    sub_chosen = []
    
    
    
    main_mode.train()
    
    batch_count = 1
    
    for x, y in main_train_loader:
        
        batch_count += 1
        
        x = x.to(device)
        y = y.to(device)
        
        
        optimizer1.zero_grad()
        
        
        
        pred = main_mode(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer1.step()
        
        main_train_losses.append(loss.item())
        main_losses.append(loss.item())
        
        
        
       
    
        
    main_mode.eval()
    
    
    for x, y in main_validation_loader:
        
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
           
            pred = main_mode(x)
            
            pred = pred.cpu()
            
            y = y.cpu()

            main_test_targets.append(y.numpy())
            
            main_test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            main_test_losses.append(loss.item())
    
    
   
    
    
    

    sub_mode.train()
    
    for x, y in sub_train_loader:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer2.zero_grad()
        
        
        
        pred = sub_mode(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer2.step()
        
        sub_train_losses.append(loss.item())
        sub_losses.append(loss.item())
        
        
        
        
    sub_mode.eval()
    
    for x, y in sub_validation_loader:
        
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():

            pred = sub_mode(x)

            pred = pred.cpu()
            y = y.cpu()

            sub_test_targets.append(y.numpy())
            sub_test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            sub_test_losses.append(loss.item())
    
    
    
    
    
main_unsu_targets = list(main_unsu_targets)
main_unsu_pred_class = list(main_unsu_pred_class)
    
for m, n in main_unsu_loader:
    n = torch.Tensor(n)
    m = m.to(device)
    n = n.to(device)

    with torch.no_grad():

        pred = main_mode(m)

        pred = pred.cpu()
        n = n.cpu()
        m = m.cpu()    
        
        main_unsu_targets.append(pred.numpy())
        main_unsu_pred_class.append(np.argmax(pred, axis=1))
    
        
        
main_unsu_targets = np.concatenate(main_unsu_targets).squeeze()
main_unsu_pred_class = np.concatenate(main_unsu_pred_class).squeeze() 
  
    
sub_unsu_targets = list(sub_unsu_targets)
sub_unsu_pred_class = list(sub_unsu_pred_class)
    
for m, n in sub_unsu_loader:
    n = torch.Tensor(n)
    m = m.to(device)
    n = n.to(device)

    with torch.no_grad():

        pred = sub_mode(m)

        pred = pred.cpu()        
        n = n.cpu()
        m = m.cpu()
        sub_unsu_targets.append(pred.numpy())
        sub_unsu_pred_class.append(np.argmax(pred, axis=1))
        
sub_unsu_targets = np.concatenate(sub_unsu_targets).squeeze()
sub_unsu_pred_class = np.concatenate(sub_unsu_pred_class).squeeze()     
        
main_test_loss = np.mean(main_test_losses)
main_test_targets = np.concatenate(main_test_targets).squeeze()




            
    
            
              


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [03:43<00:00, 223.87s/it]


ValueError: need at least one array to concatenate

In [19]:
main_test_pred_class = np.concatenate(main_test_pred_class).squeeze()
main_f1 = f1_score(main_test_targets, main_test_pred_class, average='micro')
main_test_f1.append(main_f1)
sub_test_loss = np.mean(sub_test_losses)
sub_test_targets = np.concatenate(sub_test_targets).squeeze()
sub_test_pred_class = np.concatenate(sub_test_pred_class).squeeze()
sub_f1 = f1_score(sub_test_targets, sub_test_pred_class, average='micro')
sub_test_f1.append(sub_f1)
    
print()
print('Main category losses: train - {:.3f}, test - {:.3f}'.format(np.mean(main_train_losses), main_test_loss))
print('Sub category losses: train - {:.3f}, test - {:.3f}'.format(np.mean(sub_train_losses), sub_test_loss))
print('F1 test - {:.3f}'.format(main_f1*0.7 + sub_f1*0.3))
mean_test_loss = np.mean(main_test_losses)*0.7 + np.mean(sub_test_losses)*0.3    


Main category losses: train - 1.801, test - 1.758
Sub category losses: train - 3.423, test - 3.336
F1 test - 0.432


In [20]:
main_y_unsu.update(pd.Series(list(main_unsu_pred_class)))
sub_y_unsu.update(pd.Series(list(sub_unsu_pred_class)))

In [21]:
new_main_x_train = main_x_train.append(main_x_unsu)
new_main_y_train = main_y_train.append(main_y_unsu)
new_sub_x_train = sub_x_train.append(sub_x_unsu)
new_sub_y_train = sub_y_train.append(sub_y_unsu)

In [22]:
full_main_x = new_main_x_train
full_main_y = new_main_y_train
full_sub_x = new_sub_x_train
full_sub_y = new_sub_y_train

In [23]:
def list_full(full):
    list_full_result = []
    for i in full:
        if i != '':
            list_full_result.append(i)
        else:
            list_full_result.append(0)
    
    return list_full_result

In [24]:
list_full_main_y = list_full(full_main_y)
list_full_sub_y = list_full(full_sub_y)
list_full_main_x = list_full(full_main_x)
list_full_sub_x = list_full(full_sub_x)

KeyboardInterrupt: 

In [ ]:
full_main_x = pd.Series(list_full_main_x)
full_main_y = pd.Series([i for i in list_full_main_y])
full_sub_x = pd.Series(list_full_sub_x)
full_sub_y = pd.Series([i for i in list_full_sub_y])

In [ ]:
new_main_x_test = full_main_x[int(len(full_main_x)*0.9):]
new_main_y_test = full_main_y[int(len(full_main_y)*0.9):]
new_sub_x_test = full_sub_x[int(len(full_sub_x)*0.9):]
new_sub_y_test = full_sub_y[int(len(full_sub_y)*0.9):]

new_main_x_train = full_main_x[:int(len(full_main_x)*0.9)]
new_main_y_train = full_main_y[:int(len(full_main_y)*0.9)]
new_sub_x_train = full_sub_x[:int(len(full_sub_x)*0.9)]
new_sub_y_train = full_sub_y[:int(len(full_sub_y)*0.9)]

new_main_x_test = new_main_x_test.rename("question") 
new_main_x_train = new_main_x_train.rename("question") 
new_main_y_test = new_main_y_test.rename("main_category") 
new_main_y_train = new_main_y_train.rename("main_category") 
new_sub_x_test = new_sub_x_test.rename("question") 
new_sub_x_train = new_sub_x_train.rename("question") 
new_sub_y_test = new_sub_y_test.rename("sub_category") 
new_sub_y_train = new_sub_y_train.rename("sub_category") 

In [ ]:
new_sub_y_train = new_sub_y_train.astype('int64')
new_main_y_train = new_main_y_train.astype('int64')

In [ ]:
new_main_train_dataset = WordData(list(new_main_x_train), list(new_main_y_train), word2index)
new_main_train_loader = DataLoader(new_main_train_dataset, batch_size=64)

new_sub_train_dataset = WordData(list(new_sub_x_train), list(new_sub_y_train), word2index)
new_sub_train_loader = DataLoader(new_sub_train_dataset, batch_size=64)

new_main_validation_dataset = WordData(list(new_main_x_test), list(new_main_y_test), word2index)
new_main_validation_loader = DataLoader(new_main_validation_dataset, batch_size=64)

new_sub_validation_dataset = WordData(list(new_sub_x_test), list(new_sub_y_test), word2index)
new_sub_validation_loader = DataLoader(new_sub_validation_dataset, batch_size=64)


In [ ]:
main_num_classes = 28
sub_num_classes = 209
main_mode = DeepAverageNetwork(embedding_matrix=vectors, n_classes=main_num_classes)


criterion = nn.CrossEntropyLoss()

sub_mode = DeepAverageNetwork(embedding_matrix=vectors, n_classes=sub_num_classes)
sub_mode = sub_mode.to(device)
optimizer2 = optim.Adam(sub_mode.parameters(), lr=0.01)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')    



main_mode = main_mode.to(device)

optimizer1 = optim.Adam(main_mode.parameters(), lr=0.01)

criterion = criterion.to(device)
# Training

epochs = 10
main_losses = []
sub_losses = []
best_test_loss = 10.

main_test_f1, sub_test_f1 = [], []

for n_epoch in tqdm(range(epochs)):
    
    main_train_losses = []
    main_test_losses = []
    main_test_targets = []
    main_test_pred_class = []
    sub_train_losses = []
    sub_test_losses = []
    sub_test_targets = []
    sub_test_pred_class = []
    
    sub_unsu_targets = []
    sub_unsu_pred_class = []
    main_unsu_targets = []
    main_unsu_pred_class = []
    
    
    
    
    main_mode.train()
    
    
    
    for x, y in new_main_train_loader:
        
        x = x.long()
        y = y.long()
        x = x.to(device)
        y = y.to(device)
        
        
        optimizer1.zero_grad()
        
        
        
        pred = main_mode(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer1.step()
        
        main_train_losses.append(loss.item())
        main_losses.append(loss.item())
        
        
        
        
    
        
    main_mode.eval()
    
    
    for x, y in new_main_validation_loader:
        
        x = x.long()
        y = y.long()
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
           
            pred = main_mode(x)
            
            pred = pred.cpu()
            y = y.cpu()

            main_test_targets.append(y.numpy())
            main_test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            main_test_losses.append(loss.item())
    
    
 
    
    
    

    sub_mode.train()
    
    for x, y in new_sub_train_loader:
        
        x = x.long()
        y = y.long()
        x = x.to(device)
        y = y.to(device)
        
        optimizer2.zero_grad()
        
        
        
        pred = sub_mode(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer2.step()
        
        sub_train_losses.append(loss.item())
        sub_losses.append(loss.item())
        
        
        
        
        
    sub_mode.eval()
    
    for x, y in new_sub_validation_loader:
        
        x = x.long()
        y = y.long()
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():

            pred = sub_mode(x)

            pred = pred.cpu()
            y = y.cpu()

            sub_test_targets.append(y.numpy())
            sub_test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            sub_test_losses.append(loss.item())
    

    
   
        
main_test_loss = np.mean(main_test_losses)
main_test_targets = np.concatenate(main_test_targets).squeeze()
main_test_pred_class = np.concatenate(main_test_pred_class).squeeze()
main_f1 = f1_score(main_test_targets, main_test_pred_class, average='micro')
main_test_f1.append(main_f1)
sub_test_loss = np.mean(sub_test_losses)
sub_test_targets = np.concatenate(sub_test_targets).squeeze()
sub_test_pred_class = np.concatenate(sub_test_pred_class).squeeze()
sub_f1 = f1_score(sub_test_targets, sub_test_pred_class, average='micro')
sub_test_f1.append(sub_f1)
    
print()
print('Main category losses: train - {:.3f}, test - {:.3f}'.format(np.mean(main_train_losses), main_test_loss))
print('Sub category losses: train - {:.3f}, test - {:.3f}'.format(np.mean(sub_train_losses), sub_test_loss))
print('F1 test - {:.3f}'.format(main_f1*0.7 + sub_f1*0.3))
mean_test_loss = np.mean(main_test_losses)*0.7 + np.mean(sub_test_losses)*0.3    

In [ ]:
progress_bar = tqdm(total=len(main_train_loader.dataset), desc='Epoch Main {}'.format(n_epoch + 1))

Main category losses: train - 1.719, test - 1.736
<br>Sub category losses: train - 3.306, test - 3.321
<br>F1 test - 0.438

The simple network yields a result which is slightly worse than that of the complicated network.
